In [6]:
!pip install torch>=2.0.1 safetensors==0.3.1 sentencepiece>=0.1.97 ninja==1.11.1

In [1]:
!pip install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu118

Looking in indexes: https://download.pytorch.org/whl/nightly/cu118


In [2]:
!git clone https://github.com/turboderp/exllama.git

Cloning into 'exllama'...
remote: Enumerating objects: 1246, done.
remote: Counting objects: 100% (589/589), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 1246 (delta 504), reused 455 (delta 454), pack-reused 657
Receiving objects: 100% (1246/1246), 860.00 KiB | 6.28 MiB/s, done.
Resolving deltas: 100% (888/888), done.


In [3]:
%cd /content/exllama

/content/exllama


In [4]:
!pwd

/content/exllama


In [7]:
from model import ExLlama, ExLlamaCache, ExLlamaConfig
from tokenizer import ExLlamaTokenizer
from generator import ExLlamaGenerator
from lora import ExLlamaLora
import os, glob
import torch

In [8]:
!pip install transformers > /dev/null

In [10]:
!git lfs install
!git clone https://huggingface.co/tloen/alpaca-lora-7b

Updated git hooks.
Git LFS initialized.
Cloning into 'alpaca-lora-7b'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48
Unpacking objects: 100% (48/48), 6.14 KiB | 241.00 KiB/s, done.


In [11]:
!git clone https://huggingface.co/Neko-Institute-of-Science/LLaMA-7B-4bit-128g

Cloning into 'LLaMA-7B-4bit-128g'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Unpacking objects: 100% (23/23), 479.33 KiB | 6.75 MiB/s, done.


In [12]:
# Directory containt model, tokenizer, generator

model_directory = "/content/exllama/LLaMA-7B-4bit-128g/"

# Directory containing LoRA config and weights

lora_directory = "/content/exllama/alpaca-lora-7b/"

In [13]:
# Locate files we need within those directories

tokenizer_path = os.path.join(model_directory, "tokenizer.model")
model_config_path = os.path.join(model_directory, "config.json")
st_pattern = os.path.join(model_directory, "*.safetensors")
model_path = glob.glob(st_pattern)[0]

In [14]:
lora_config_path = os.path.join(lora_directory, "adapter_config.json")
lora_path = os.path.join(lora_directory, "adapter_model.bin")

# Create config, model, tokenizer and generator

config = ExLlamaConfig(model_config_path)               # create config from config.json
config.model_path = model_path                          # supply path to model weights file

model = ExLlama(config)                                 # create ExLlama instance and load the weights
tokenizer = ExLlamaTokenizer(tokenizer_path)            # create tokenizer from tokenizer model file

cache = ExLlamaCache(model)                             # create cache for inference
generator = ExLlamaGenerator(model, tokenizer, cache)   # create generator

In [15]:
# Load LoRA

lora = ExLlamaLora(model, lora_config_path, lora_path)

# Configure generator

generator.settings.token_repetition_penalty_max = 1.2
generator.settings.temperature = 0.65
generator.settings.top_p = 0.4
generator.settings.top_k = 0
generator.settings.typical = 0.0

In [19]:
prompt = \
    "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n" \
    "\n" \
    "### Instruction:\n" \
    "List five colors in alphabetical order.\n" \
    "\n" \
    "### Response:"

# Generate with LoRA

print(" --- LoRA ----------------- ")
print("")

generator.lora = lora
torch.manual_seed(1337)

 --- LoRA ----------------- 



In [17]:
output = generator.generate_simple(prompt, max_new_tokens = 200)
print(output)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
List five colors in alphabetical order.

### Response:
Brown, Green, Orange, Purple, Red


In [31]:
prompt = """
From Wikipedia, the free encyclopedia

A large language model (LLM) is a computerized language model consisting of an artificial neural network with many parameters (tens of millions to billions), trained on large quantities of unlabeled text containing up to trillions of tokens, using self-supervised learning or semi-supervised learning achieved by parallel computing.[1] With advent of transformers, notable for requiring less training time compared to older long short-term memory (LSTM) models,[2] thus enabling large (language) datasets, such as the Wikipedia Corpus and Common Crawl, to be used for training due to parallelization,[3] LLMs emerged around 2018. They are general purpose models, excelling at a wide range of tasks. The level of proficiency exhibited by language models in various tasks, as well as the breadth of tasks they can handle, rely less on the model's design and more on the size of the training corpus, the quantity of parameters, and the computational power achieved by parallel computing.[4] This has shifted the focus of natural language processing research away from the previous paradigm of training specialized supervised models for specific tasks.[5]

Because the training corpus and number of parameters were so extensive, despite being trained on simple tasks such as predicting the next word in a sentence, they have along the way (implicitly) learned syntax and semantics of human language. Although they acquired general "knowledge" about the world, inherent in the statements contained in the training corpus, they also acquired the inaccuracies and biases from those statements.[5]
Training

Most LLM are pre-trained such that given a training dataset of text tokens, the model predicts the tokens in the dataset. There are two general styles of such pretraining:[6]

    autoregressive (GPT-style, "predict the next word"): Given a segment of text like "I like to eat" the model predicts the next tokens, like "ice cream".
    masked ("BERT-style",[7] "cloze test"): Given a segment of text like "I like to [MASK] [MASK] cream" the model predicts the masked tokens, like "eat ice".

LLMs may be trained on auxiliary tasks which test their understanding of the data distribution, such as Next Sentence Prediction (NSP), in which pairs of sentences are presented and the model must predict whether they appear consecutively in the training corpus.[7]

Usually, LLMs are trained to minimize a specific loss function: the average negative log likelihood per token (also called cross-entropy loss).[citation needed] For example, if an autoregressive model, given "I like to eat", predicts a probability distribution P r ( ⋅ | I like to eat ) {\displaystyle Pr(\cdot |{\text{I like to eat}})} then the negative log likelihood loss on this token is − log ⁡ P r ( ice | I like to eat ) {\displaystyle -\log Pr({\text{ice}}|{\text{I like to eat}})}.

During training, regularization loss is also used to stabilize training. However regularization loss is usually not used during testing and evaluation. There are also many more evaluation criteria than just negative log likelihood. See the section below for details.
Training dataset size

The earliest LLMs were trained on corpora having on the order of billions of words.

GPT-1, the first model in OpenAI's numbered series of generative pre-trained transformer models, was trained in 2018 on BookCorpus, consisting of 985 million words.[8] In the same year, BERT was trained on a combination of BookCorpus and English Wikipedia, totalling 3.3 billion words.[7] Since then, training corpora for LLMs have increased by orders of magnitude, reaching up to trillions of tokens.[7]
Training cost

LLMs are computationally expensive to train. A 2020 study estimated the cost of training a 1.5 billion parameter model (2 orders of magnitude smaller than the state of the art at the time) at anywhere from $80 thousand to $1.6 million.[9][10] Advances in software and hardware have brought the cost substantially down, with a 2023 paper reporting a cost of 72,300 A100-GPU-hours to train a 12 billion parameter model.[11]

For Transformer-based LLM, training cost is much higher than inference cost. It costs 6 FLOPs per parameter to train on one token, whereas it costs 1 to 2 FLOPs per parameter to infer on one token.[12]

Companies in the 2020s invested large sums into increasingly large LLMs. GPT-2 (1.5 billion parameters, 2019) cost $50,000 to train, while Google PaLM (540 billion parameters, 2022) cost $8 million to train.[13]
Application to downstream tasks

Between 2018 and 2020, the standard method for harnessing an LLM for a specific natural language processing (NLP) task was to fine tune the model with additional task-specific training. It has subsequently been found that more powerful LLMs such as GPT-3 can solve tasks without additional training via "prompting" techniques, in which the problem to be solved is presented to the model as a text prompt, possibly with some textual examples of similar problems and their solutions.[5]
Fine-tuning
Main article: Fine-tuning (machine learning)

Fine-tuning is the practice of modifying an existing pretrained language model by training it (in a supervised fashion) on a specific task (e.g. sentiment analysis, named-entity recognition, or part-of-speech tagging). It is a form of transfer learning. It generally involves the introduction of a new set of weights connecting the final layer of the language model to the output of the downstream task. The original weights of the language model may be "frozen", such that only the new layer of weights connecting them to the output are learned during training. Alternatively, the original weights may receive small updates (possibly with earlier layers frozen).[7]
Prompting
See also: Prompt engineering and Few-shot learning (natural language processing)

In the prompting paradigm, popularized by GPT-3,[14] the problem to be solved is formulated via a text prompt, which the model must solve by providing a completion (via inference). In "few-shot prompting", the prompt includes a small number of examples of similar (problem, solution) pairs.[5] For example, a sentiment analysis task of labelling the sentiment of a movie review could be prompted as follows:[14]

Review: This movie stinks.
Sentiment: negative

Review: This movie is fantastic!
Sentiment:

If the model outputs "positive", then it has correctly solved the task. In zero-shot prompting, no solved examples are provided.[9][15] An example of a zero-shot prompt for the same sentiment analysis task would be "The sentiment associated with the movie review 'This movie is fantastic!' is".[16]

Few-shot performance of LLMs has been shown to achieve competitive results on NLP tasks, sometimes surpassing prior state-of-the-art fine-tuning approaches. Examples of such NLP tasks are translation, question answering, cloze tasks, unscrambling words, and using a novel word in a sentence.[15] The creation and optimisation of such prompts is called prompt engineering.
Instruction tuning

Often, instruction tuning is necessary because otherwise an artificial neural network, in response to user 's instruction "Write an essay about the main themes represented in Hamlet," may generate a response such as "If you submit the essay after March 17th, your grade will be reduced by 10% for each day of delay" based on the frequency of this textual sequence in the corpus. It is only through instruction tuning that the model learns what the response should actually contain for specific instructions.

Answer the following question

How are LLMs pretrained?
"""

# Generate with LoRA

print(" --- LoRA ----------------- ")
print("")

generator.lora = lora
torch.manual_seed(1337)

 --- LoRA ----------------- 



In [32]:
output = generator.generate_simple(prompt, max_new_tokens = 200)
print(output)

 
From Wikipedia, the free encyclopedia

A large language model (LLM) is a computerized language model consisting of an artificial neural network with many parameters (tens of millions to billions), trained on large quantities of unlabeled text containing up to trillions of tokens, using self-supervised learning or semi-supervised learning achieved by parallel computing.[1] With advent of transformers, notable for requiring less training time compared to older long short-term memory (LSTM) models,[2] thus enabling large (language) datasets, such as the Wikipedia Corpus and Common Crawl, to be used for training due to parallelization,[3] LLMs emerged around 2018. They are general purpose models, excelling at a wide range of tasks. The level of proficiency exhibited by language models in various tasks, as well as the breadth of tasks they can handle, rely less on the model's design and more on the size of the training corpus, the quantity of parameters, and the computational power achiev

In [34]:
prompt = """From Wikipedia, the free encyclopedia

A large language model (LLM) is a computerized language model consisting of an artificial neural network with many parameters (tens of millions to billions), trained on large quantities of unlabeled text containing up to trillions of tokens, using self-supervised learning or semi-supervised learning achieved by parallel computing.[1] With advent of transformers, notable for requiring less training time compared to older long short-term memory (LSTM) models,[2] thus enabling large (language) datasets, such as the Wikipedia Corpus and Common Crawl, to be used for training due to parallelization,[3] LLMs emerged around 2018. They are general purpose models, excelling at a wide range of tasks. The level of proficiency exhibited by language models in various tasks, as well as the breadth of tasks they can handle, rely less on the model's design and more on the size of the training corpus, the quantity of parameters, and the computational power achieved by parallel computing.[4] This has shifted the focus of natural language processing research away from the previous paradigm of training specialized supervised models for specific tasks.[5]

Because the training corpus and number of parameters were so extensive, despite being trained on simple tasks such as predicting the next word in a sentence, they have along the way (implicitly) learned syntax and semantics of human language. Although they acquired general "knowledge" about the world, inherent in the statements contained in the training corpus, they also acquired the inaccuracies and biases from those statements.[5]
Training

Most LLM are pre-trained such that given a training dataset of text tokens, the model predicts the tokens in the dataset. There are two general styles of such pretraining:[6]

    autoregressive (GPT-style, "predict the next word"): Given a segment of text like "I like to eat" the model predicts the next tokens, like "ice cream".
    masked ("BERT-style",[7] "cloze test"): Given a segment of text like "I like to [MASK] [MASK] cream" the model predicts the masked tokens, like "eat ice".

LLMs may be trained on auxiliary tasks which test their understanding of the data distribution, such as Next Sentence Prediction (NSP), in which pairs of sentences are presented and the model must predict whether they appear consecutively in the training corpus.[7]

Usually, LLMs are trained to minimize a specific loss function: the average negative log likelihood per token (also called cross-entropy loss).[citation needed] For example, if an autoregressive model, given "I like to eat", predicts a probability distribution P r ( ⋅ | I like to eat ) {\displaystyle Pr(\cdot |{\text{I like to eat}})} then the negative log likelihood loss on this token is − log ⁡ P r ( ice | I like to eat ) {\displaystyle -\log Pr({\text{ice}}|{\text{I like to eat}})}.

During training, regularization loss is also used to stabilize training. However regularization loss is usually not used during testing and evaluation. There are also many more evaluation criteria than just negative log likelihood. See the section below for details.
Training dataset size

The earliest LLMs were trained on corpora having on the order of billions of words.

GPT-1, the first model in OpenAI's numbered series of generative pre-trained transformer models, was trained in 2018 on BookCorpus, consisting of 985 million words.[8] In the same year, BERT was trained on a combination of BookCorpus and English Wikipedia, totalling 3.3 billion words.[7] Since then, training corpora for LLMs have increased by orders of magnitude, reaching up to trillions of tokens.[7]
Training cost

LLMs are computationally expensive to train. A 2020 study estimated the cost of training a 1.5 billion parameter model (2 orders of magnitude smaller than the state of the art at the time) at anywhere from $80 thousand to $1.6 million.[9][10] Advances in software and hardware have brought the cost substantially down, with a 2023 paper reporting a cost of 72,300 A100-GPU-hours to train a 12 billion parameter model.[11]

For Transformer-based LLM, training cost is much higher than inference cost. It costs 6 FLOPs per parameter to train on one token, whereas it costs 1 to 2 FLOPs per parameter to infer on one token.[12]

Companies in the 2020s invested large sums into increasingly large LLMs. GPT-2 (1.5 billion parameters, 2019) cost $50,000 to train, while Google PaLM (540 billion parameters, 2022) cost $8 million to train.[13]
Application to downstream tasks

Between 2018 and 2020, the standard method for harnessing an LLM for a specific natural language processing (NLP) task was to fine tune the model with additional task-specific training. It has subsequently been found that more powerful LLMs such as GPT-3 can solve tasks without additional training via "prompting" techniques, in which the problem to be solved is presented to the model as a text prompt, possibly with some textual examples of similar problems and their solutions.[5]
Fine-tuning
Main article: Fine-tuning (machine learning)

Fine-tuning is the practice of modifying an existing pretrained language model by training it (in a supervised fashion) on a specific task (e.g. sentiment analysis, named-entity recognition, or part-of-speech tagging). It is a form of transfer learning. It generally involves the introduction of a new set of weights connecting the final layer of the language model to the output of the downstream task. The original weights of the language model may be "frozen", such that only the new layer of weights connecting them to the output are learned during training. Alternatively, the original weights may receive small updates (possibly with earlier layers frozen).[7]
Prompting
See also: Prompt engineering and Few-shot learning (natural language processing)

In the prompting paradigm, popularized by GPT-3,[14] the problem to be solved is formulated via a text prompt, which the model must solve by providing a completion (via inference). In "few-shot prompting", the prompt includes a small number of examples of similar (problem, solution) pairs.[5] For example, a sentiment analysis task of labelling the sentiment of a movie review could be prompted as follows:[14]

Review: This movie stinks.
Sentiment: negative

Review: This movie is fantastic!
Sentiment:

If the model outputs "positive", then it has correctly solved the task. In zero-shot prompting, no solved examples are provided.[9][15] An example of a zero-shot prompt for the same sentiment analysis task would be "The sentiment associated with the movie review 'This movie is fantastic!' is".[16]

Few-shot performance of LLMs has been shown to achieve competitive results on NLP tasks, sometimes surpassing prior state-of-the-art fine-tuning approaches. Examples of such NLP tasks are translation, question answering, cloze tasks, unscrambling words, and using a novel word in a sentence.[15] The creation and optimisation of such prompts is called prompt engineering.
Instruction tuning

Often, instruction tuning is necessary because otherwise an artificial neural network, in response to user 's instruction "Write an essay about the main themes represented in Hamlet," may generate a response such as "If you submit the essay after March 17th, your grade will be reduced by 10% for each day of delay" based on the frequency of this textual sequence in the corpus. It is only through instruction tuning that the model learns what the response should actually contain for specific instructions.

Answer the following question
What was the estimate of the 2020 study"""

output = generator.generate_simple(prompt, max_new_tokens = 200)
print(output)

From Wikipedia, the free encyclopedia

A large language model (LLM) is a computerized language model consisting of an artificial neural network with many parameters (tens of millions to billions), trained on large quantities of unlabeled text containing up to trillions of tokens, using self-supervised learning or semi-supervised learning achieved by parallel computing.[1] With advent of transformers, notable for requiring less training time compared to older long short-term memory (LSTM) models,[2] thus enabling large (language) datasets, such as the Wikipedia Corpus and Common Crawl, to be used for training due to parallelization,[3] LLMs emerged around 2018. They are general purpose models, excelling at a wide range of tasks. The level of proficiency exhibited by language models in various tasks, as well as the breadth of tasks they can handle, rely less on the model's design and more on the size of the training corpus, the quantity of parameters, and the computational power achieved

In [ ]:
# Generate without LoRA

print("")
print(" --- No LoRA -------------- ")
print("")

generator.lora = None

torch.manual_seed(1337)

output = generator.generate_simple(prompt, max_new_tokens = 200)
print(output)